## Tiny Encryption Algorithm

In [10]:
import base64
import ctypes
import itertools
import math

In [28]:
def _str2vec(value):
    l = 4
    n = len(value)
    num_chunks = math.ceil(n/1)
    chunks = [value[l * i:l * (i +1)]
              for i in range(num_chunks)]
    result = [sum([character << 8 * j
                 for j, character in enumerate(chunk)])
              for chunk in chunks]
    return result
def _vec2str(vector, l=4):
    result = bytes((element >> 8 * i) & 0xff
                 for element in vector
                 for i in range(l)).replace(b'\x00', b'')
    return result

In [34]:
def _encipher(m, k):
    y, z = [ctypes.c_uint32(x)
            for x in m]
    sum = ctypes.c_uint32(0)
    delta = 0x9E3779B9
    
    for n in range(32, 0, -1):
        sum.value += delta
        y.value += (z.value << 4) + k[0] ^ z.value + sum.value ^ (z.value >> 5) + k[1]
        z.value += (y.value << 4) + k[2] ^ y.value + sum.value ^ (y.value >> 5) + k[3]
    return [y.value, z.value]

In [35]:
def _decipher(v, k):
    y, z = [ctypes.c_uint32(x)
            for x in v]
    sum = ctypes.c_uint32(0xC6EF3720)
    delta = 0x9E3779B9

    for n in range(32, 0, -1):
        z.value -= (y.value << 4) + k[2] ^ y.value + sum.value ^ (y.value >> 5) + k[3]
        y.value -= (z.value << 4) + k[0] ^ z.value + sum.value ^ (z.value >> 5) + k[1]
        sum.value -= delta

    return [y.value, z.value]

In [36]:
def _chunks(iterable, n):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk

In [37]:
def encrypt(plaintext, key):
    v = _str2vec(plaintext.encode())
    k = _str2vec(key.encode()[:16])
    bytearray = b''.join(_vec2str(_encipher(chunk, k))for chunk in _chunks(v, 2))
    return base64.b64encode(bytearray).decode()


In [40]:
print(encrypt('hello','0123456789abcdef'))

ValueError: not enough values to unpack (expected 2, got 1)